<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/Saliency_Maps_CatDogsDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download test files and weights



In [ ]:
# Download the same test files from the Cats vs Dogs ungraded lab
!wget -O cat1.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/cat1.jpeg
!wget -O cat2.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/cat2.jpeg
!wget -O catanddog.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/catanddog.jpeg
!wget -O dog1.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/dog1.jpeg
!wget -O dog2.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/dog2.jpeg

# Download prepared weights
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kipXTxesGJKGY1B8uSPRvxROgOH90fih' -O 0_epochs.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oiV6tjy5k7h9OHGTQaf0Ohn3FmF-uOs1' -O 15_epochs.h5


### Import the required packages



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
import cv2

### Download and prepare the dataset.



#### Load Cats vs Dogs Dataset



In [ ]:
# Load the data and create the train set (optional: val and test sets)
train_data = tfds.load('cats_vs_dogs',split = 'train[:80%]',as_supervised = True)
validation_data = tfds.load('cats_vs_dogs',split = 'train[:80%]',as_supervised = True )
test_data = tfds.load('cats_vs_dogs',split = 'train[80%:90%]',as_supervised = True)
# YOUR CODE HERE

#### Create preprocessing function

Define a function that takes in an image and label. This will:
  * cast the image to float32
  * normalize the pixel values to [0, 1]
  * resize the image to 300 x 300


In [ ]:
def augment_images(image, label):
  # YOUR CODE HERE
  image = tf.cast(image,tf.float32)

  # Normalize the pixel values
  image = image / 255.0

  # Resize to 300 x 300
  image = tf.image.resize(image,(300,300))

  return image, label

#### Preprocess the training set



In [ ]:
augmented_training_data = train_data.map(augment_images)

#### Create batches of the training set.



In [ ]:
train_batches = augmented_training_data.batch(32)

### Build the Cats vs Dogs classifier



In [ ]:
model = tf.keras.models.Sequential()
model.add(Conv2D(16,input_shape = (300,300,3),kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(GlobalAveragePooling2D())
model.add(Dense(2,activation = 'softmax'))

model.summary()

### Create a function to generate the saliency map



In [ ]:
def do_salience(image, model, label, prefix):

  # Read the image and convert channel order from BGR to RGB
  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Resize the image to 300 x 300 and normalize pixel values to the range [0, 1]
  img = cv2.resize(img, (300, 300)) / 255.0

  # Add an additional dimension (for the batch), and save this in a new variable
  exp_image = np.expand_dims(img, axis=0)

  # Declare the number of classes
  num_classes = 2

  # Define the expected output array by one-hot encoding the label
  # The length of the array is equal to the number of classes
  expected_output = tf.one_hot([label] * exp_image.shape[0], num_classes)

  # Witin the GradientTape block:
  # Cast the image as a tf.float32
  # Use the tape to watch the float32 image
  # Get the model's prediction by passing in the float32 image
  # Compute an appropriate loss
  # between the expected output and model predictions.
  # you may want to print the predictions to see if the probabilities adds up to 1

  with tf.GradientTape() as tape:
      inputs = tf.cast(exp_image, dtype=tf.float32)
      tape.watch(inputs)
      prediction = model(inputs)
      loss = tf.keras.losses.categorical_crossentropy(expected_output, prediction)

      print(prediction)

  # get the gradients of the loss with respect to the model's input image

  gradients = tape.gradient(loss, inputs)

  # generate the grayscale tensor
  grayscale_tensor = tf.reduce_sum(tf.abs(gradients), axis = -1)

  # normalize the pixel values to be in the range [0, 255].
  # the max value in the grayscale tensor will be pushed to 255.
  # the min value will be pushed to 0.
  # Use the formula: 255 * (x - min) / (max - min)
  # Use tf.reduce_max, tf.reduce_min
  # Cast the tensor as a tf.uint8

  normalized_tensor = tf.cast(
    255 * (grayscale_tensor - tf.reduce_min(grayscale_tensor)) / (tf.reduce_max(grayscale_tensor) - tf.reduce_min(grayscale_tensor)),
    tf.uint8)


  # Remove dimensions that are size 1
  normalized_tensor = tf.squeeze(normalized_tensor)


  # plot the normalized tensor
  # Set the figure size to 8 by 8
  # do not display the axis
  # use the 'gray' colormap
  # This code is provided for you.
  plt.figure(figsize=(8, 8))
  plt.axis('off')
  plt.imshow(normalized_tensor, cmap='gray')
  plt.show()

  # optional: superimpose the saliency map with the original image, then display it.
  # we encourage you to do this to visualize your results better
  gradient_color = cv2.applyColorMap(normalized_tensor.numpy(), cv2.COLORMAP_HOT)
  gradient_color = gradient_color / 255.0
  super_imposed = cv2.addWeighted(img, 0.7, gradient_color, 0.3, 0.0)

  plt.figure(figsize=(8, 8))
  plt.imshow(super_imposed)
  plt.axis('off')
  plt.show()


  # save the normalized tensor image to a file. this is already provided for you.
  salient_image_name = prefix + image
  normalized_tensor = tf.expand_dims(normalized_tensor, -1)
  normalized_tensor = tf.io.encode_jpeg(normalized_tensor, quality=100, format='grayscale')
  writer = tf.io.write_file(salient_image_name, normalized_tensor)

### Generate saliency maps with untrained model



In [ ]:
# load initial weights
model.load_weights('0_epochs.h5')

# generate the saliency maps for the 5 test images
# YOUR CODE HERE
img_list = ['cat1.jpg', 'cat2.jpg', 'catanddog.jpg', 'dog1.jpg', 'dog2.jpg']
img_label = [0,0,0,1,1]
for i,l in zip(img_list,img_label):
  do_salience(i,model,l,'epoch0_salient')

With untrained weights, you will see something like this in the output.
- You will see strong pixels outside the cat that the model uses that when classifying the image.
- After training that these will slowly start to localize to features inside the pet.

<img src='https://drive.google.com/uc?export=view&id=1h5wP52lwbBUMVLlsgyb-tQl_I9eu42X7' alt='saliency'>


### Configure the model for training



In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

### Train your model



In [ ]:
# load pre-trained weights
model.load_weights('15_epochs.h5')

# train the model for just 3 epochs
history = model.fit(train_batches,epochs = 3)

### Generate saliency maps at 18 epochs



In [ ]:
for i,l in zip(img_list,img_label):
  do_salience(i,model,l,'salient')